In [25]:
! pip install sklearn

ERROR: Could not find a version that satisfies the requirement pickle
ERROR: No matching distribution found for pickle


In [26]:
import pandas as pd
import numpy as np
import sklearn
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import KFold


In [2]:
data = pd.read_csv("cleaned_credit_data.csv")

In [3]:
data.head()

Unnamed: 0 chk_acct  duration credit_his purpose  amount saving_acct  \
0           0      A11         6        A34     A43    1169         A65   
1           1      A12        48        A32     A43    5951         A61   
2           2      A14        12        A34     A46    2096         A61   
3           3      A11        42        A32     A42    7882         A61   
4           4      A11        24        A33     A40    4870         A61   

  present_emp  installment_rate  sex  ... property  age other_install  \
0         A75                 4  A93  ...     A121   67          A143   
1         A73                 2  A92  ...     A121   22          A143   
2         A74                 2  A93  ...     A121   49          A143   
3         A74                 2  A93  ...     A122   45          A143   
4         A73                 3  A93  ...     A124   53          A143   

   housing n_credit   job  n_people telephone foreign response  
0     A152        2  A173      good      A192    A201        0  
1     A152        1  A173      good      A191    A201        1  
2     A152        1  A172       bad      A191    A201        0  
3     A153        1  A173       bad      A191    A201        0  
4     A153        2  A173       bad      A191    A201        1  

[5 rows x 22 columns]

In [4]:
predict = "response"

X = data[["sex", "housing", "saving_acct", "chk_acct", "age", "duration", "amount"]]
y = np.array(data[predict])

In [5]:
X.head()

sex housing saving_acct chk_acct  age  duration  amount
0  A93    A152         A65      A11   67         6    1169
1  A92    A152         A61      A12   22        48    5951
2  A93    A152         A61      A14   49        12    2096
3  A93    A153         A61      A11   45        42    7882
4  A93    A153         A61      A11   53        24    4870

In [6]:
#ONE HOT ENCODING THE CATEGORICAL DATA AND NORMALIZE THE NUMERICAL DATA

In [7]:
#One hot encoding
ohe = make_column_transformer((OneHotEncoder(), ["sex", "housing", "saving_acct", "chk_acct"]), remainder="passthrough")
x = ohe.fit_transform(X)

#Normalize to 0 to 1
normalize = MinMaxScaler()
normalize.fit(x)
x = normalize.transform(x)

In [20]:
print(x)

[[0.         0.         1.         ... 0.85714286 0.02941176 0.05056674]
 [0.         1.         0.         ... 0.05357143 0.64705882 0.31368989]
 [0.         0.         1.         ... 0.53571429 0.11764706 0.10157368]
 ...
 [0.         0.         1.         ... 0.33928571 0.11764706 0.03048311]
 [0.         0.         1.         ... 0.07142857 0.60294118 0.08776274]
 [0.         0.         1.         ... 0.14285714 0.60294118 0.23803235]]


In [9]:
#DIVIDING DATA INTO TRAIN, VALIDATION, AND TES 0.8-0.1-0.1

In [10]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.1)

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train, y_train, test_size=(0.1/0.9))


In [11]:
#LOGISTIC REGRESSION

In [12]:
logisticReg = linear_model.LogisticRegression().fit(x_train, y_train)
logisticReg.score(x_test, y_test)

0.73

In [13]:
y_predicted = logisticReg.predict(x_test)
confusion_matrix(y_test, y_predicted)

array([[60,  7],
       [20, 13]], dtype=int64)

In [14]:
#The number of true positive is 64; false positive is 5; false negative is 18; and true negative is 13 

In [24]:
#Logistic Regression, SVC, Random Forest

In [31]:
kf = KFold(n_splits=10)

acc_logisticReg = 0
acc_svc = 0
acc_ranForest = 0

for train_index, test_index in kf.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    logisticReg = linear_model.LogisticRegression()
    logisticReg.fit(x_train, y_train)
    acc1 = logisticReg.score(x_test, y_test)

    svc = svm.SVC()
    svc.fit(x_train, y_train)
    acc2 = svc.score(x_test, y_test)
    
    ranForest = RandomForestClassifier(max_depth=20)
    ranForest.fit(x_train, y_train)
    acc3 = ranForest.score(x_test, y_test)
    
    if acc1 > acc_logisticReg:
        acc_logisticReg = acc1
        with open("logisticReg.pickle", "wb") as f:
            pickle.dump(logisticReg, f)
    
    if acc2 > acc_svc:
        acc_svc = acc2
        with open("SVC.pickle", "wb") as f:
            pickle.dump(svc, f)
            
    if acc3 > acc_ranForest:
        acc_ranForest = acc3
        with open("ranForest.pickle", "wb") as f:
            pickle.dump(ranForest, f)

In [32]:
print("Accuracy of Logistic Regression: ", acc_logisticReg)
print("Accuracy of SVC: ", acc_svc)
print("Accuracy of Random Forest: ", acc_ranForest)

Accuracy of Logistic Regression:  0.82
Accuracy of SVC:  0.81
Accuracy of Random Forest:  0.79


In [33]:
#Load models from pickle

In [34]:
pickle_in = open("logisticReg.pickle", "rb")
logisticReg = pickle.load(pickle_in)

pickle_in = open("SVC.pickle", "rb")
svc = pickle.load(pickle_in)

pickle_in = open("ranForest.pickle", "rb")
ranForest = pickle.load(pickle_in)